In [1]:
import matplotlib.pyplot as plt
from collections import Counter
import glob2
import pandas as pd
from tqdm.notebook import tqdm
import h5py
import numpy as np

plt.ion()
plt.show()
% load_ext autoreload
% autoreload 2


UsageError: Line magic function `%` not found.


In [2]:
from scipy.stats import skew, kurtosis
from matplotlib.backends.backend_pdf import PdfPages

In [6]:
# category = "real"
category = "imbalanced"
# category = "balanced"
path = "."
if category in ["balanced", "imbalanced"]:
    files = glob2.glob(f'{path}/{category}/*.h5')
    files = [f[len(f"{path}/{category}/"):-3] for f in files]
else:
    files = glob2.glob(f'{path}/real/*.h5')
    files = [f[len(f"{path}/real/"):-3] for f in files]
print(files)

['data_-0.5c0.6', 'data_-0.5c0.7', 'data_-0.5c0.8', 'data_-0.5c0.9', 'data_-0.5c1', 'data_-1c0.6', 'data_-1c0.7', 'data_-1c0.8', 'data_-1c0.9', 'data_-1c1', 'data_0.5c0.6', 'data_0.5c0.7', 'data_0.5c0.8', 'data_0.5c0.9', 'data_0.5c1', 'data_0c0.6', 'data_0c0.7', 'data_0c0.8', 'data_0c0.9', 'data_0c1', 'data_1c0.6', 'data_1c0.7', 'data_1c0.8', 'data_1c0.9', 'data_1c1']


In [7]:
df = pd.DataFrame(columns=["dataset", "size", "nb_clusters",
                           "sparsity", "non_zero_skew",
                           "non_zero_kurosis", "max_value", "non_zero_mean", "non_zero_median", "dropout_rate",
                           "cluster_sizes", ])
pdf = PdfPages(f'./output/{category}_data_distribution.pdf')
for dataset in tqdm(files):
    if category in ["balanced", "imbalanced"]:
        data_mat = h5py.File(f"{path}/{category}/{dataset}.h5", "r")
    else:
        data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    Y = np.array(data_mat['Y'])
    X = np.array(data_mat['X'])
    dropout_rate = np.array(data_mat['dropout'])
    print(f">>>>dataset {dataset}")

    if category == "real" and X.shape[0] > 20000:
        continue
    n_clusters = np.unique(Y).shape[0]
    sparsity = len(np.where(X == 0)[0]) / (X.shape[0] * X.shape[1])
    # X = np.log1p(X)
    max_value = int(np.max(X))
    non_zero = X[np.where(X != 0)]
    non_zero_mean = np.mean(non_zero)
    non_zero_median = np.median(non_zero)
    non_zero_skew = skew(non_zero)
    non_zero_kurosis = kurtosis(non_zero)
    df.loc[df.shape[0]] = [
        dataset, f"{X.shape[0]} x {X.shape[1]}", n_clusters,

        sparsity, non_zero_skew, non_zero_kurosis, max_value, non_zero_mean, non_zero_median, dropout_rate,
        list(zip(*Counter(Y).most_common()))[1]]

    fig = plt.figure()
    plt.hist(non_zero, bins=30, log=True)
    plt.xlabel("expression values")
    plt.ylabel("log counts")
    plt.title(dataset)
    plt.close()
    pdf.savefig(fig)

pdf.close()
df.sort_values(by="nb_clusters").reset_index(
    drop=True).to_csv(f"./output/description_{category}.csv")

  0%|          | 0/25 [00:00<?, ?it/s]

>>>>dataset data_-0.5c0.6
>>>>dataset data_-0.5c0.7
>>>>dataset data_-0.5c0.8
>>>>dataset data_-0.5c0.9
>>>>dataset data_-0.5c1
>>>>dataset data_-1c0.6
>>>>dataset data_-1c0.7
>>>>dataset data_-1c0.8
>>>>dataset data_-1c0.9
>>>>dataset data_-1c1
>>>>dataset data_0.5c0.6
>>>>dataset data_0.5c0.7
>>>>dataset data_0.5c0.8
>>>>dataset data_0.5c0.9
>>>>dataset data_0.5c1
>>>>dataset data_0c0.6
>>>>dataset data_0c0.7
>>>>dataset data_0c0.8
>>>>dataset data_0c0.9
>>>>dataset data_0c1
>>>>dataset data_1c0.6
>>>>dataset data_1c0.7
>>>>dataset data_1c0.8
>>>>dataset data_1c0.9
>>>>dataset data_1c1
